In [1]:
import os
import sys

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
fig_dir = os.path.join(project_dir, 'fig')
data_dir = os.path.join(project_dir, 'data')
log_dir = os.path.join(project_dir, 'log')
os.makedirs(fig_dir, exist_ok=True)

# Add the src directory to sys.path
sys.path.append(src_dir)

import mech.full_DPSGD as DPSGDModule

In [2]:
data_args = {
    "method": "default",
    "data_dir": data_dir,
    "internal_result_path": "/scratch/bell/wei402/fdp-estimation/results"
}

args = DPSGDModule.generate_params(data_args=data_args, log_dir=log_dir, model_type="CNN")
sampler = DPSGDModule.DPSGDSampler(args)

Files already downloaded and verified


05/15/2025 12:09:19:INFO:Initialized CNN_DPSGDSampler with parameters: batch_size=512, epochs=1, lr=0.10, sigma=1.00, max_grad_norm=1.00, device=cpu


In [5]:
score0, score1 = sampler.preprocess(num_samples=6)

05/15/2025 12:10:06:INFO:Found 5 existing model pairs. Need to generate 1 more.
05/15/2025 12:10:06:INFO:Model loaded from /scratch/bell/wei402/fdp-estimation/results/model_folder/CNN_model_x0_bf2ef00babcfc3fb.pt
05/15/2025 12:10:06:INFO:Model loaded from /scratch/bell/wei402/fdp-estimation/results/model_folder/CNN_model_x1_846663d5fa347774.pt
05/15/2025 12:10:06:INFO:Loaded and projected model pair 1/5
05/15/2025 12:10:06:INFO:Model loaded from /scratch/bell/wei402/fdp-estimation/results/model_folder/CNN_model_x0_b510dc3c365a9a30.pt
05/15/2025 12:10:06:INFO:Model loaded from /scratch/bell/wei402/fdp-estimation/results/model_folder/CNN_model_x1_3cee69a762c3495e.pt
05/15/2025 12:10:06:INFO:Loaded and projected model pair 2/5
05/15/2025 12:10:06:INFO:Model loaded from /scratch/bell/wei402/fdp-estimation/results/model_folder/CNN_model_x0_dc33417a6e37b103.pt
05/15/2025 12:10:06:INFO:Model loaded from /scratch/bell/wei402/fdp-estimation/results/model_folder/CNN_model_x1_efeef71487fae47a.pt


In [6]:
print(score0)
print(score1)

[[3.60289559e-05]
 [1.13770552e-03]
 [6.49817241e-03]
 [4.95460874e-04]
 [4.46706964e-03]
 [2.24081939e-03]]
[[0.00104067]
 [0.00075073]
 [0.00048683]
 [0.00081685]
 [0.00135131]
 [0.00531252]]


In [9]:
import multiprocessing


def _train_model_worker(args):
    """Worker function for parallel model training."""
    sampler_kwargs, positive = args
    import torch
    torch.set_num_threads(1)
    sampler = DPSGDModule.DPSGDSampler(sampler_kwargs)
    model, model_path = sampler.train_model(positive=positive)
    return model_path


def parallel_train_models(sampler_kwargs, num_generating_samples=32):
    """Train num_generating_samples models in parallel, each on a single CPU thread."""
    with multiprocessing.Pool(processes=num_generating_samples) as pool:
        # Prepare arguments: half for positive=False, half for positive=True
        args_list = [(sampler_kwargs, False)] * num_generating_samples + [(sampler_kwargs, True)] * num_generating_samples
        results = pool.map(_train_model_worker, args_list)
    return results

# Example usage:
sampler_kwargs = DPSGDModule.generate_params(data_args=data_args, log_dir=log_dir, model_type="CNN")
model_paths = parallel_train_models(sampler_kwargs, num_generating_samples=32)

Files already downloaded and verified


05/15/2025 12:32:47:INFO:Initialized CNN_DPSGDSampler with parameters: batch_size=512, epochs=1, lr=0.10, sigma=1.00, max_grad_norm=1.00, device=cpu
/tmp/fdp-env/lib/python3.8/site-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
  0%|          | 0/98 [00:00<?, ?it/s]/tmp/fdp-env/lib/python3.8/site-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
  1%|          | 1/98 [00:01<01:46,  1.10s/it]05/15/2025 12:32:48:INFO:Initialized CNN_DPS

In [10]:
data_args = {
    "method": "default",
    "data_dir": data_dir,
    "internal_result_path": "/scratch/bell/wei402/fdp-estimation/results"
}

args = DPSGDModule.generate_params(data_args=data_args, log_dir=log_dir, model_type="CNN")
sampler = DPSGDModule.CNN_DPSGDSampler(args)
score0, score1 = sampler.preprocess(num_samples=20)

print(score0)
print(score1)

Files already downloaded and verified


05/15/2025 12:37:29:INFO:Initialized CNN_DPSGDSampler with parameters: batch_size=512, epochs=1, lr=0.10, sigma=1.00, max_grad_norm=1.00, device=cpu
05/15/2025 12:37:29:INFO:Found 22 existing model pairs. Need to generate 0 more.
/home/wei402/Desktop/fdp-estimation/src/mech/full_DPSGD.py:210: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_o

[[3.60289559e-05]
 [4.04671620e-04]
 [4.04671620e-04]
 [1.13770552e-03]
 [6.49817241e-03]
 [4.95460874e-04]
 [4.04671620e-04]
 [4.04671620e-04]
 [4.04671620e-04]
 [4.46706964e-03]
 [4.04671620e-04]
 [4.04671620e-04]
 [4.04671620e-04]
 [4.04671620e-04]
 [4.04671620e-04]
 [4.04671620e-04]
 [3.20070656e-03]
 [4.04671620e-04]
 [1.99789787e-03]
 [4.04671620e-04]]
[[0.00040467]
 [0.00040467]
 [0.00040467]
 [0.00040467]
 [0.00104067]
 [0.00040467]
 [0.00040467]
 [0.00531252]
 [0.00075073]
 [0.00048683]
 [0.00081685]
 [0.00040467]
 [0.00040467]
 [0.00040467]
 [0.00040467]
 [0.00040467]
 [0.00040467]
 [0.00040467]
 [0.00040467]
 [0.00040467]]
